In [1]:
# importing required modules
import PyPDF2
import re
import pandas as pd
import os
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# folder_loc = input("Enter folder location:")
folder_loc = "/Users/nilaygaitonde/Documents/E-RevBay/Automation/data"
# Create a list of all files in the folder
files = os.listdir(folder_loc)
# Create a list of all files with .pdf extension
pdf_files = [f for f in files if f.endswith('.pdf')]

In [3]:
def get_index(firstIndex,string):
    return int(firstIndex)+int(len(string))

In [4]:
def save_as_csv(data_df,pivot_df,csv,filename):
    if(csv):
        data_df.to_csv(f'{filename}_1.csv',index=False)
        pivot_df.to_csv(f'{filename}_2.csv',index=True)
    else:
        with pd.ExcelWriter(f'{filename}.xlsx') as writer:
            data_df.to_excel(writer,sheet_name='Sheet1',index=False)
            pivot_df.to_excel(writer,sheet_name='Sheet2',index=True)

In [5]:
folder_loc = input("Enter folder location:")
files = os.listdir(folder_loc)
pdf_files = [f for f in files if f.endswith('.pdf')]

In [6]:
# folder_loc = input("Enter folder location:")
# files = os.listdir(folder_loc)
# pdf_files = [f for f in files if f.endswith('.pdf')]
# complete_String = ""
# for file in pdf_files:
#     print("For name",file)
#     salary = int(input("enter salary:"))
#     pdfFileObj = open(folder_loc+"/"+file, 'rb')
# pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
# for i in range(pdfReader.numPages):
#     pageObj = pdfReader.getPage(i).extract_text()
#     complete_String += pageObj

In [7]:
def create_loan(text,completeDF):
    completeDF = {"type":[],"institution":[],"date_opened":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
    countAcc = text.count("Acct # :")
    for _ in range(countAcc):
        delinquecy = False
        accountNoIndex = text.find("Acct # :")
        text = text[accountNoIndex+1:]
        openIndex = get_index(text.find('Open: '),'Open: ')
        openValue = text[openIndex:(text.find("Date Reported: "))].strip()

        dateOpenedIndex = get_index(text.find('Date Opened: '),'Date Opened: ')
        dateOpenedValue = text[dateOpenedIndex:(text.find("Type: "))].strip()
        try:
            date_object = datetime.strptime(dateOpenedValue, '%d-%m-%Y').date()
        except ValueError:
            pass
        # print(text)
        balanceIndex = get_index(text.find('Balance: '),'Balance: ')
        balance = text[balanceIndex:(text.find('Open:'))]
        if(balance[0] == "R"):
            balance = balance[4:]
        balance = int(balance.replace(',',''))
        # Find institution
        instiutionIndex = get_index(text.find('Institution : '),'Institution : ')
        instiutionName = text[instiutionIndex:(text.find('Past Due Amount'))-1]

        # Find type of loan
        typeIndex = get_index(text.find('Type: '),'Type: ')
        typeName = text[typeIndex:(text.find('Last Payment:'))-1]

        # TODO:::::::: FILL NAMES OF LOAN TYPES

        if(typeName not in ["Personal Loan","Business Loan","Credit Card","Housing Loan","Auto Loan",""]):
            typeName = "4_Others"
        elif(typeName == "Credit Card"):
            typeName = "1_CreditCard"
        elif(typeName == "Personal Loan"):
            typeName = "3_PersonalLoan"
        elif(typeName == "Business Loan"):
            typeName = "4_BusinessLoan"
        # Find EMI
        emiIndex = get_index(text.find('Monthly Payment Amount: '),'Monthly Payment Amount:')
        emiValue = text[emiIndex:(text.find('Credit Limit:'))-1]
        if(emiValue == ''):
            emiValue = 0
        else:
            emiValue = emiValue[4:]
            try:
                emiValue = int(emiValue.replace(',',''))
            except ValueError:
                emiValue = 0
        
        if(typeName == '1_CreditCard'):
            creditIndex = get_index(text.find('Credit Limit:'),"Credit Limit: Rs. ")
            creditLimit = text[creditIndex:text.find('Collateral Value')-1]
            try:
                sanction_credit = int(creditLimit.replace(',',''))
            except ValueError:
                sanction_credit = 0
            emiValue = balance*0.05
        else:
            sanctionIndex = get_index(text.find('Sanction Amount :'),"Sanction Amount : ")
            sanctionLimit = text[sanctionIndex:text.find('Reason:')]
            # print(sanctionLimit)
            if(sanctionLimit == ''):
                sanction_credit = 0
            else:
                sanctionLimit = sanctionLimit[4:]
                try:
                    sanction_credit = int(sanctionLimit.replace(',',''))
                except ValueError:
                    sanction_credit = 0
        AccountIndex = get_index(text.find('Account Status: '),'Account Status: ')
        AccountStatus = text[AccountIndex:text.find('Asset Classification')].strip()
        if(AccountStatus in [" ","Closed Account","Standard","Current Account"]):
            delinquecy = False
        else:
            delinquecy = True
        completeDF['balance'].append(int(balance))
        completeDF['institution'].append(instiutionName.strip())
        completeDF['type'].append(typeName.strip())
        completeDF['sanction_credit'].append(int(sanction_credit))
        completeDF['emi'].append(int(emiValue))
        completeDF['paid_principle'].append(int(sanction_credit-balance))
        completeDF['open'].append(openValue)
        completeDF['delinquecy'].append(delinquecy)
        completeDF['date_opened'].append(date_object)
    return completeDF

In [8]:
# finalDF = {"type":[],"institution":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
# for file in pdf_files:
#     print("For name",file)
#     salary = input("enter salary:")
#     pdfFileObj = open(folder_loc+file, 'rb')
#     pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#     for i in range(pdfReader.numPages):
#         pageObj = pdfReader.getPage(i).extract_text()
#         on_page_2  = True
#         balance = 0
#         if(i == 0):
#             try:
#                 pageObj.index("Account Details :")
#                 on_page_2 = False
#             except ValueError:
#                 on_page_2 = True
#         if not on_page_2:
#             balanceIndex = get_index(pageObj.find('Balance: '),'Balance: ')
#             balance = pageObj[balanceIndex:(pageObj.find('Open:'))]
#             if(balance[0] == "R"):
#                 balance = balance[4:]
#             balance = int(balance.replace(',',''))
#         newDF = create_loan(pageObj,on_page_2,finalDF,balance)
#         [finalDF["type"].append(i) for i in newDF["type"]]
#         [finalDF["institution"].append(i) for i in newDF["institution"]]
#         [finalDF["sanction_credit"].append(i) for i in newDF["sanction_credit"]]
#         [finalDF["balance"].append(i) for i in newDF["balance"]]
#         [finalDF["emi"].append(i) for i in newDF["emi"]]
#         [finalDF["paid_principle"].append(i) for i in newDF["paid_principle"]]
#         [finalDF['open'].append(i) for i in newDF["open"]]
#         [finalDF['delinquecy'].append(i) for i in newDF["delinquecy"]]
        

In [9]:
finalDF = {"type":[],"institution":[],"date_opened":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
complete_String = ""
for file in pdf_files:
    salary = int(input(f"For name,{file}: \nEnter salary:"))
    pdfFileObj = open(folder_loc+"/"+file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    for i in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(i).extract_text()
        complete_String += pageObj
    riskIndex = get_index(complete_String.find("Equifax Risk Score 3.1 "), "Equifax Risk Score 3.1 ")
    riskValue = int(complete_String[riskIndex:(complete_String.find("1. "))].strip())
    print(riskValue)
    newDF = create_loan(complete_String,finalDF)
    [finalDF["type"].append(i) for i in newDF["type"]]
    [finalDF["institution"].append(i) for i in newDF["institution"]]
    [finalDF["sanction_credit"].append(i) for i in newDF["sanction_credit"]]
    [finalDF["balance"].append(i) for i in newDF["balance"]]
    [finalDF["emi"].append(i) for i in newDF["emi"]]
    [finalDF["paid_principle"].append(i) for i in newDF["paid_principle"]]
    [finalDF['open'].append(i) for i in newDF["open"]]
    [finalDF['delinquecy'].append(i) for i in newDF["delinquecy"]]
    [finalDF['date_opened'].append(i) for i in newDF["date_opened"]]
    data_df = pd.DataFrame.from_dict(finalDF)
    data_df['open'] = data_df['open'].map({
        'Yes': True,
        'No': False
    })
    data_df = data_df.loc[data_df['open']==True]
    # data_df.drop(['open'],axis=1,inplace=True)
    data_df.sort_values(by=['type'],ascending=True,inplace=True)
    # balance = 0 and no delinquecy is false
    # make filter for balance = 0 and delinquecy = false
    data_df = data_df.loc[(data_df['balance']!=0) | (data_df['delinquecy']==True)]
    data_df['paid_principle'] = data_df['paid_principle'].apply(lambda x: 0 if x<0 else x)
    data_df.groupby('type').sum()
    # salary = 50000
    df1=pd.DataFrame()
    if(salary <= 50000):
        FOIR = salary*0.5
    elif(salary > 50000 and salary <= 150000):
        FOIR = salary*0.6
    else:
        FOIR = salary*0.7
    delinquency = data_df['delinquecy'].sum()
    if(delinquency>0):
        delinquency = True
    else:
        delinquency = False
    disposable = FOIR - data_df['emi'].sum()
    print(disposable,FOIR,data_df['emi'].sum())
    sum_df = {
        'type': 'Total',
        'sanction_credit': data_df['sanction_credit'].sum(),
        'balance': data_df['balance'].sum(),
        'emi': data_df['emi'].sum(),
        'paid_principle': data_df['paid_principle'].sum(),
        'delinquecy': data_df['delinquecy'].sum(),
        'foir': FOIR,
        'disposable': disposable
    }
    sum_df = pd.DataFrame(sum_df,index=[0])
    pivot_df = data_df.groupby('type').sum()
    pivot_df = pivot_df.reset_index()
    pivot_df = pivot_df.append(sum_df,ignore_index=True)
    pivot_df.set_index('type',inplace=True)
    data_df.set_index('type',inplace=True)
    filename = os.path.basename(file).split('.')[0]
    save_as_csv(data_df = data_df,pivot_df = pivot_df,csv = True,filename=filename)

755
12500.0 12500.0 0


/var/folders/b4/9cg_tkrn3256x3hcybpxqnpw0000gn/T/ipykernel_62735/1626500506.py:35: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_df.groupby('type').sum()
/var/folders/b4/9cg_tkrn3256x3hcybpxqnpw0000gn/T/ipykernel_62735/1626500506.py:62: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivot_df = data_df.groupby('type').sum()
/var/folders/b4/9cg_tkrn3256x3hcybpxqnpw0000gn/T/ipykernel_62735/1626500506.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pivot_df = pivot_df.append(sum_df,ignore_index=True)


In [10]:
# disp = prq
# q = a/b
# a = (1 + (0.15/12))**60
var1 = 1 + (0.15/12)
var2 = 0.15/12
var3 = var1**60

value1 = (var3 - 1)*disposable
print(disposable)
value2 = var2*var3
principal_amt = value1/value2
print(principal_amt)

12500.0
525432.3974313625


In [11]:
# -------------------OUTPUT-------------------
print("FOIR: ",FOIR)
print("emi",data_df['emi'].sum())
print("Disposable: ",disposable)
print("Delinuency:",delinquency)
# if(delinquency):
#     print("You have delinuency in your loan and you are not eligible for loan")
# else:
#     if(disposable>0):
#         print("You are eligible for loan")
#     else:
#         print("You are not eligible for loan")

FOIR:  12500.0
emi 0
Disposable:  12500.0
Delinuency: False
